In [ ]:
import tensorflow as tf
assert tf.__version__ >= "2.0"

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

## Load NLP model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine", use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

## Run widget

In [ ]:
#@title
import ipywidgets as widgets
from IPython.display import display

class Color:   
   GREEN = '\033[92m'
   RED = '\033[91m'
   BOLD = '\033[1m'   
   END = '\033[0m'

button = widgets.Button(
    description='CLASSIFY !',
    button_style='primary'
  )

text_area = widgets.Textarea(
    value='',
    placeholder='Type something',
    description='',
    disabled=False
)
output = widgets.Output()

def on_button_clicked(b):
  text = text_area.value
  result = nlp(text)
  print(f'results: {result}')
  prediction = result[0]["label"]

  if prediction == "POSITIVE":    
    color = Color.GREEN    
  else:
    color = Color.RED

  with output:    
    print(Color.BOLD + color + f'{prediction}: ' + Color.END + f'"{text}"')

button.on_click(on_button_clicked)
display(text_area, button, output)
